In [40]:
import os
import pandas as pd
from ai21 import AI21Client
from ai21.models.chat import ChatMessage, ResponseFormat
from multiprocessing import Pool, cpu_count
import numpy as np
import time
import random

In [125]:


# Retrieve the API key from environment variable
api_key = os.getenv('AI21LABS_API_KEY')

# Check if the API key is set
if not api_key:
    raise ValueError("API key not found. Please set the AI21_API_KEY environment variable.")

# Set up the AI21 client
client = AI21Client(api_key=api_key)

def generate_clinical_summary(row):
    """Generate a clinical summary based on the provided row of data."""
    # Build context string from the DataFrame row
    context = "\n".join(f"{col}: {row[col]}" for col in df1_filtered.columns if pd.notna(row[col]))

    # Define the prompt
    prompt = f"""You are a helpful assistant that generates clinical notes using medical terminology.
    Utilize reasoning to analyze the following structured data and provide a detailed, narrative clinical note
    reflecting typical communication in clinical notes without suggesting solutions. Use paragraph breaks to separate different aspects of the note.
    Data: {context}

    <<Sample output of Clinical Note>>:
    The patient, with ID 10005817, was administered a single dose of IV antibiotics as per the treatment plan. The administration occurred on December 16, 2132, at 19:50, with a total volume of 500 ml delivered.

    The patient weighs 91 kg and was under the care of caregiver ID 4793. The status of the medication administration is noted as "Finished Running."
    
    Please provide a similar style of clinical note for the given data."""

    # Create the chat completion
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=[ChatMessage(role="user", content=prompt)],
        n=1,
        max_tokens=1024,
        temperature=0.4,
        top_p=1,
        response_format=ResponseFormat(type="text"),
    )


    # Extract and return the generated summary
    return response.choices[0].message.content


In [6]:
# Load the CSV file into a DataFrame
df = pd.read_csv('datasets/mimic/icu/inputevents.csv')

In [8]:
df1= pd.read_csv('datasets/mimic/icu/procedureevents.csv')

In [151]:
# Find a subject_id that is present in both df and df1
subject_id = df['subject_id'].iloc[2]

# Filter df1 for the same subject_id
df_filtered = df[df['subject_id'] == subject_id]
df1_filtered = df1[df1['subject_id'] == subject_id]

# Now you can proceed with your processing

In [130]:
# Save df_filtered to a CSV file
df_filtered.to_csv('datasets/single_patient/inputevents_filtered.csv', index=False)
df1_filtered.to_csv('datasets/single_patient/procedureevents_filtered.csv', index=False)

In [48]:
num_processes = 10

In [71]:
generate_clinical_summary(df_filtered.iloc[1])

'**Clinical Note:**\n\nThe patient, with ID 10005817, was administered a dose of antibiotics as per the treatment plan. The administration occurred on December 15, 2132, at 20:15, with the medication recorded as finished running by 20:16. The total volume delivered was 1.0 dose.\n\nThe patient weighs 91.0 kg and was under the care of caregiver ID 92805. The status of the medication administration is noted as "FinishedRunning."'

In [117]:
# Define the number of parallel processes

# Split the DataFrame into chunks
df_chunks = np.array_split(df_filtered, num_processes)

# Function to process a chunk of the DataFrame
def process_chunk(chunk):
    # Introduce a random delay
    time.sleep(random.uniform(0, 1))
    
    for index, row in chunk.iterrows():
        chunk.at[index, 'clinical_summary'] = generate_clinical_summary(row)
        print(".", end="", flush=True)
    #chunk['clinical_summary'] = chunk.apply(generate_clinical_summary, axis=1)
    return chunk

# Use multiprocessing to apply the function to all rows of the DataFrame
with Pool(num_processes) as pool:
    results = pool.map(process_chunk, df_chunks)

# Combine the results
df_processed = pd.concat(results)


/home/andrei/build/analytiq/patient-timeline-detection/.venv/lib64/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


TypeError: generate_clinical_summary() missing 1 required positional argument: 'row'

In [55]:
# Save the updated DataFrame to a new CSV file
output_file = 'datasets/single_patient/inputevents_filtered_with_summary2.csv'
df_processed.to_csv(output_file, index=False)

print(f"Clinical summaries generated and saved to {output_file}")

Clinical summaries generated and saved to datasets/single_patient/inputevents_filtered_with_summary.csv


In [78]:
df1_filtered.keys()

Index(['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'starttime',
       'endtime', 'storetime', 'itemid', 'value', 'valueuom', 'location',
       'locationcategory', 'orderid', 'linkorderid', 'ordercategoryname',
       'ordercategorydescription', 'patientweight', 'isopenbag',
       'continueinnextdept', 'statusdescription', 'ORIGINALAMOUNT',
       'ORIGINALRATE'],
      dtype='object')

In [153]:
# Create an "amount" column populated with 0
generate_clinical_summary(df1_filtered.iloc[0])

'**Clinical Note:**\n\nThe patient, with ID 10005817, was placed on mechanical ventilation as per the treatment plan. The ventilation started on January 6, 2135, at 08:34 and ended on January 19, 2135, at 14:00. The total duration of ventilation was 19046 minutes.\n\nThe patient weighs 88.8 kg and was under the care of caregiver ID 99850. The order category for this treatment is classified under "Ventilation" with a description of "ContinuousProcess." The status of the ventilation is noted as "FinishedRunning." The original amount and rate of ventilation were recorded as 19046.0 and 1, respectively.'

In [154]:
df1_filtered_chunks = np.array_split(df1_filtered, num_processes)

/home/andrei/build/analytiq/patient-timeline-detection/.venv/lib64/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [155]:
# Use multiprocessing to apply the function to all rows of the DataFrame
with Pool(num_processes) as pool:
    results = pool.map(process_chunk, df1_filtered_chunks)

# Combine the results
df1_processed = pd.concat(results)

.............................................

In [158]:
# Save the updated DataFrame to a new CSV file
output_file = 'datasets/single_patient/procedureevents_filtered_with_summary3.csv'
df1_processed.to_csv(output_file, index=False)

print(f"Clinical summaries generated and saved to {output_file}")

Clinical summaries generated and saved to datasets/single_patient/procedureevents_filtered_with_summary3.csv


In [148]:
def generate_clinical_overall_summary(df):
    """Generate a clinical summary based on the provided row of data."""
    # Dump the clinical_summary column into a string
    context = "\n".join(df.clinical_summary.to_string())

    # Define the prompt
    prompt = f"""You are a helpful assistant that generates clinical notes using medical terminology.
    Utilize reasoning to analyze the following structured data and provide a detailed, narrative clinical note
    reflecting typical communication in clinical notes without suggesting solutions. Use paragraph breaks to separate different aspects of the note.
    Data: {context}

    Please provide a similar style of clinical note for the given data.
    
    Include dates and times in the narrative. Highlight the most important procedures
    """

    # Create the chat completion
    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=[ChatMessage(role="user", content=prompt)],
        n=1,
        max_tokens=4048,
        temperature=0.4,
        top_p=1,
        response_format=ResponseFormat(type="text"),
    )


    # Extract and return the generated summary
    return response.choices[0].message.content

In [152]:
generate_clinical_overall_summary(df1_processed)

'Clinical Note:\n\nThe patient, with ID 10005817, underwent a procedure on 2023-05-15 at 10:00 AM. The procedure involved the administration of medication, specifically 500 mg of Azithromycin, which was given intravenously. The patient was monitored for any adverse reactions post-administration.\n\nOn 2023-05-16 at 08:00 AM, the patient was administered a second dose of Azithromycin, maintaining the same dosage and route of administration. Vital signs were closely monitored, and no immediate adverse reactions were noted.\n\nThe patient, with ID 10005817, was administered a third dose of Azithromycin on 2023-05-17 at 08:00 AM, continuing the same dosage and route. The patient remained stable, with no significant changes in vital signs or adverse reactions reported.\n\nThe patient, with ID 10005817, was discharged on 2023-05-18 at 12:00 PM after completing the course of Azithromycin. The patient was advised to follow up with their primary care physician within the next week.'

In [159]:
# Save to a file
with open('datasets/single_patient/procedureevents_filtered_with_summary3_overall.txt', 'w') as f:
    f.write(generate_clinical_overall_summary(df1_processed))
